# stores

> The `Reax` stores implementation became its own package, [Sveltish](fredguth.github.io/sveltish).  Here I use it and add some customizations for our present use case.

## Setup

In [1]:
#|hide
%env XLA_PYTHON_CLIENT_MEM_FRACTION=0.2
%load_ext autoreload
%autoreload 2

env: XLA_PYTHON_CLIENT_MEM_FRACTION=0.2


In [2]:
#| default_exp stores

## Using sveltish stores

`Reax` uses the concept of a `Store`, an observable object to which you can subscribe to `call you back`(callback) when it changes. It [became its own package](fredguth.github.io/sveltish) where it is better documented.

Here we will use it and customize it.

There are 3 kinds of stores:
- `Writable stores` have a `set` method, so you can change their value;
- `Readable stores` are read-only (they can only be set by code given in its initialization);
- `Derived stores` are stores whose values derive from other stores.

Example:

In [3]:
from sveltish.stores import writable, readable, derived
import threading
import time

In [20]:
utc = writable(-3) # writable store
utc

w<0> $int: -3

Notice that the store representation is `w<0> $int: -3`, where `w<0>` stands for a writable store with 0 subscribers. The `$int` is a type hint for the store value.

In [22]:
def publisher(set): # the start notifier function
    stopped = threading.Event()
    def loop(): # needs to be in a separate thread
        while not stopped.wait(1): # in seconds
            set(time.gmtime())
    threading.Thread(target=loop).start()    
    return stopped.set
clock = readable(time.gmtime(), publisher) # readable store
clock

r<0> $struct_time: time.struct_time(tm_year=2023, tm_mon=3, tm_mday=8, tm_hour=15, tm_min=8, tm_sec=32, tm_wday=2, tm_yday=67, tm_isdst=0)

In [23]:
stop = clock.subscribe(lambda x: print(time.strftime(f"%H:%M:%S", x)))
time.sleep(2);stop()

15:08:32
15:08:44
15:08:45


In [24]:
utc_clock = derived([clock, utc], lambda clock, utc: tuple(clock)[:3] + (tuple(clock)[3]+utc,) + tuple(clock)[4:])
stop = utc_clock.subscribe(lambda x: print(time.strftime(f"%H:%M:%S", x)))
time.sleep(2);stop()

12:08:45
12:08:46
12:08:47


Another interesting aspect of `sveltish stores` is its `pipe` operator:

In [6]:
a =  writable(0)
stop1 = (a| (lambda x: x+1)).subscribe(lambda x: print(f'callback 1: {x}'))

callback 1: 1


In [7]:
a.set(2)

callback 1: 3


In [8]:
stop2 = (a| (lambda x: x+1)| (lambda x: x*x) ).subscribe(lambda x: print(f'callback 2: {x}'))

callback 2: 9


In [9]:
a.set(1)
a.set(3)


callback 1: 2
callback 2: 4
callback 1: 4
callback 2: 16


In [10]:
stop1(), stop2()

(None, None)

## Reax custom stores

In [11]:
#|export
from sveltish.stores import writable, StoreProtocol, Subscriber, Unsubscriber, Readable, Writable

In [12]:
#|export
class Mutable(Writable):
    def __init__(self, **kw):
        self.__clsName = kw.pop('clsName', 'dict')
        self.__store = writable(kw)
    def __call__(self): return self.__store()
    def __repr__(self) -> str:
        return f"{self.__store}".replace('dict', str(self.__clsName))
    def __str__(self) -> str:
        return str(self.__store)
    def __getattr__(self, k:str):
        try:
            return self.__store.get()[k]
        except:
            raise AttributeError(k)
    def __setattr__(self, k:str,v) -> None:
        if f'_{self.__class__.__name__}_' in k: super().__setattr__(k,v)
        else: self.__store.set(self.__store.get() | {k:v})
    def subscribe(self, callback:Subscriber) -> Unsubscriber:
        return self.__store.subscribe(callback)
    def set(self, v) -> None: return self.__store.set(v)
    def update(self, fn) -> None: return self.__store.update(fn)

In [13]:
obj = Mutable(a=1,b=2, c='foo')
str(obj)

"w<0> $dict: {'a': 1, 'b': 2, 'c': 'foo'}"

In [14]:
stop = obj.subscribe(lambda x: print(f'callback 1: {x}'))
obj.b=3
stop()
obj()

callback 1: {'a': 1, 'b': 2, 'c': 'foo'}
callback 1: {'a': 1, 'b': 3, 'c': 'foo'}


{'a': 1, 'b': 3, 'c': 'foo'}

In [15]:
class Base(dict):
    def __init__(self, params, state, run, **kwds):
        super(Base, self).__init__(params=params, state=state, run=run, **kwds)
        self.__dict__ = self

In [16]:
#|export
def mutable(value):
    return Mutable(**value, clsName=value.__class__.__name__)

In [17]:
b = Base(params={'a':1}, state={'b':2}, run=lambda x: x)

In [18]:
sb = mutable(b)
sb

w<0> $Base: {'params': {'a': 1}, 'state': {'b': 2}, 'run': <function <lambda> at 0x7f9d2a9eff70>}

In [19]:
#| hide
import nbdev; nbdev.nbdev_export()